In [2]:
# libraries
%run pre_req.py

In [3]:
# creating data for prediction
input_data = pd.DataFrame({'CreditScore' : 796,
                          'Geography' : 'France',
                          'Gender' : 'Male',
                          'Age' : 34,
                          'Tenure' : 8,
                          'Balance': 60263.23,
                           'NumOfProducts' : 3,
                          'HasCrCard' : 1,
                          'IsActiveMember' : 0,
                          'EstimatedSalary' : 170835.32,
                          },
                         index = [0])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,796,France,Male,34,8,60263.23,3,1,0,170835.32


In [4]:
# encoding geography
with open('onehot_encoder_geography.pkl', 'rb') as file:
    geo_encoder = pickle.load(file)
geo_encoder.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [5]:
encoded = geo_encoder.transform(input_data[['Geography']])
encoded_df = pd.DataFrame(encoded.toarray(), columns = geo_encoder.get_feature_names_out(['Geography']))
encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
# gender encoding
with open('label_encoder_gender.pkl', 'rb') as file:
    gender_encoder = pickle.load(file)

input_data['Gender'] = gender_encoder.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,796,France,1,34,8,60263.23,3,1,0,170835.32


In [7]:
# final input df
test_data = pd.concat([input_data.drop('Geography', axis = 1), encoded_df], axis = 1)
test_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,796,1,34,8,60263.23,3,1,0,170835.32,1.0,0.0,0.0


In [8]:
# scaling
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
    
test_data = scaler.transform(test_data)
test_data

array([[ 1.49805012,  0.91324755, -0.46564102,  1.03972843, -0.25359662,
         2.53355998,  0.64920267, -1.02583358,  1.22409166,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
# prediction
model = load_model('model.keras')
prediction = model.predict(test_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


array([[0.30613592]], dtype=float32)

In [10]:
if prediction[0] > 0.5:
    print('Will Churn')
else:
    print('Just Chill')

Just Chill
